In [1]:
import pandas as pd
import re
from nltk import flatten
import numpy as np

# Data pre-processing work related to baseline model




In [2]:
data = pd.read_csv('cleaned_tweets.csv')
emoji_data = pd.read_csv('emoji.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64383 entries, 0 to 64382
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   64383 non-null  int64 
 1   id      64383 non-null  int64 
 2   time    64383 non-null  object
 3   tweets  64383 non-null  object
 4   emoji   64383 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.5+ MB


In [4]:
emoji_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   symbol      60 non-null     object
 2   Name        60 non-null     object
 3   Score       60 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.0+ KB


In [5]:
#display(emoji_data)
#display(data)

**Formatting the input data to a desired format**

In [8]:
tweets_posts=[]
target_number=[]
target_emoji=[]
#take 250 examples for now
for e in emoji_data['symbol']:
  tweets_posts.append(data[data.emoji==e]['tweets'][:250].tolist()) #find the first 250 (subject to change) tweets of emoji e
  target_number.append([emoji_data[emoji_data['symbol']==e].index.values]*250) #append the index of emoji e 250 times (subject to change) to a list
  target_emoji.append([e]*250)

#they are nested list so need to flatten them
target_number=flatten(target_number)
target_number=[i[0] for i in target_number]
target_emoji=flatten(target_emoji)
tweets_posts=flatten(tweets_posts)

#create a dictionary
data2 = {'tweets':tweets_posts,
        'emojis_num':target_number,
         'emoji':target_emoji}

**Train Validation Split**

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
#convert dictionary to a dataframe
tweets_dataset = pd.DataFrame(data2)
display(tweets_dataset)

,tweets,emojis_num,emoji
0,"Great content! Very valuable, informative and ...",0,👏
1,Thank you Champ Calvin,0,👏
2,"Congratulations, Dr. Damborsky! Awesome coinci...",0,👏
3,Kindly help retweet my business tweet,0,👏
4,"Here's to our goals this week, let's crush the...",0,👏
...,...,...,...
14995,Less Followers? LIKE &amp; FOLLOW LIKE | | ...,59,▶️
14996,What if browsing the Internet could make you m...,59,▶️
14997,Abeg repost am now me self I don press play t...,59,▶️
14998,peace of mind ~ partynextdoor,59,▶️


In [15]:
X_train, X_valid,Y_train,Y_valid = train_test_split(data2['tweets'],data2['emojis_num'], test_size=0.2,stratify=data2['emojis_num'],random_state=3)
print(len(X_train))
print(len(X_valid))

12000
3000


In [16]:
print(Y_train)

[14, 47, 1, 3, 20, 40, 29, 36, 26, 53, 31, 50, 48, 43, 18, 47, 23, 58, 41, 31, 30, 26, 9, 56, 54, 20, 44, 12, 0, 52, 30, 28, 6, 4, 21, 14, 57, 54, 47, 41, 11, 10, 35, 2, 39, 8, 51, 31, 11, 19, 46, 42, 5, 45, 49, 20, 41, 29, 4, 46, 0, 49, 59, 31, 27, 34, 30, 59, 59, 23, 16, 23, 46, 1, 21, 2, 32, 8, 1, 11, 1, 27, 16, 53, 15, 8, 18, 34, 2, 21, 58, 32, 2, 24, 4, 4, 25, 42, 3, 33, 30, 11, 22, 43, 36, 46, 10, 13, 32, 50, 37, 57, 4, 18, 30, 47, 56, 32, 9, 39, 26, 0, 18, 49, 27, 51, 50, 57, 21, 38, 44, 54, 34, 18, 53, 18, 41, 35, 46, 36, 29, 31, 23, 50, 2, 26, 56, 55, 2, 31, 57, 5, 36, 12, 54, 49, 32, 27, 23, 15, 47, 35, 38, 12, 36, 49, 18, 25, 26, 40, 29, 29, 15, 3, 14, 39, 10, 20, 36, 27, 27, 28, 29, 49, 57, 36, 3, 54, 36, 31, 21, 8, 22, 38, 52, 7, 56, 4, 19, 47, 57, 11, 31, 45, 39, 18, 29, 33, 41, 51, 55, 36, 12, 48, 16, 7, 19, 14, 47, 4, 51, 43, 42, 1, 43, 21, 29, 16, 1, 23, 47, 23, 58, 6, 59, 46, 21, 14, 0, 59, 9, 28, 11, 24, 8, 31, 29, 7, 33, 41, 49, 48, 55, 55, 23, 17, 4, 29, 16, 45, 37

**Convert Y_data to one-hot format as required by baseline model**

In [17]:
def Y_one_hot(Y):
  j=0
  y_list=[]
  #for loop to go through all the y_data
  for i in Y: # i is the emoji index number
    t=np.zeros(60) #不是很确定用Tensor还是numpy？？？
    t[i]=1 #corresponding emoji index is 1
    y_list.append(t)
    j=j+1
  return y_list

In [18]:
Y_train=Y_one_hot(Y_train)
Y_valid=Y_one_hot(Y_valid)

In [20]:
display(Y_train)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.

# Baseline Model 

In [ ]:
#Baseline code start